# 内存地址分析

## 变量作用域与内存地址概述
在 Python 中，每个变量都存储的是**对象的引用（地址）**，而不是值本身。变量的作用域决定了它在程序中可访问的范围。
Python 中变量分为三类作用域：

|类型|	说明|	常见场景|
|----|-------|------------|
|全局变量|	定义在所有函数之外的变量，作用于整个模块	|全局配置、常量等
|局部变量	|定义在函数内部的变量，仅在函数体内可见	|函数参数、中间变量
|闭包变量	|外层函数中的局部变量，被内层函数引用但不修改	|闭包、装饰器中

## 分析内存地址与作用域


### 全局变量与局部变量

In [1]:
x = 10  # 全局变量

def func():
    x = 20  # 局部变量（与全局x无关）
    print("局部变量 x 的地址:", id(x))

func()
print("全局变量 x 的地址:", id(x))

局部变量 x 的地址: 1554715050832
全局变量 x 的地址: 1554715050512


说明：
1. id() 函数可以查看变量的内存地址
2. func() 中的 x 是一个局部变量，它与全局变量 x 无关（即使名字相同）。

1.局部作用域修改全局变量（不可以，默认会创建局部变量）

In [4]:
x = 100

def modify_global_wrong():
    x = x + 1  # ❌ 报错：UnboundLocalError
    print(x)

modify_global_wrong()

UnboundLocalError: local variable 'x' referenced before assignment

错误原因：
1. 一旦在函数内部对变量赋值，Python 会默认认为是局部变量。
2. x = x + 1 中左边的 x 是局部变量，右边的 x 还未定义 → 报错。

总结：在局部作用域中，对全局变量是**可读不可改**

2.正确修改全局变量 —— 使用 global

In [5]:
x = 100

def modify_global_right():
    global x  # 声明使用全局变量 x
    x = x + 1
    print("修改后的全局变量 x =", x)

modify_global_right()
print("最终 x =", x)

修改后的全局变量 x = 101
最终 x = 101


解释：
1. 使用 global x，告诉 Python “我用的是全局变量，不要创建局部变量”。
2. 修改成功，全局变量 x 的值也被改变了。

### 局部变量和闭包变量

1.闭包变量的形成

In [2]:
def outer():
    y = 30  # 闭包变量
    def inner():
        print("闭包变量 y 的地址:", id(y))
    return inner

f = outer()
f()

闭包变量 y 的地址: 1554715051152


说明：
1. 变量y是`outer`的局部变量。
2. 但 inner 中引用了 y，此时 y 成为一个闭包变量（即保存在函数的自由变量中）。
3. 即使 `outer()`已执行完毕，y 的值和地址依然保留在 inner 的环境中。

2.修改闭包变量时使用 nonlocal

In [3]:
def outer():
    count = 0
    def inner():
        nonlocal count
        count += 1
        print("count =", count, "地址:", id(count))
    return inner

counter = outer()
counter()
counter()

count = 1 地址: 1554715050224
count = 2 地址: 1554715050256


说明：
1. 如果不加 nonlocal，count 在 `inner()` 中会被当作新变量（导致报错）。
2. 使用 nonlocal 后，`inner()` 就可以修改外层局部变量，并保持同一个内存地址。